In [ ]:
!pip install transformers

In [ ]:
!pip install nltk

In [ ]:
!pip install catboost

In [9]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from collections import Counter
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

Глобальные параметры

In [10]:
TAG_RANGE = list(range(1,40))

TOKENIZER_DIR = ''
TOKENIZER = TOKENIZER_DIR + "DeepPavlov/rubert-base-cased"

MODEL_DIR = ''
DATASET_NAME = MODEL_DIR + ''
MODEL_NAME = MODEL_DIR + ''
PARAMS = dict()

# Модели для обучения

In [15]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

class TextVectorizerBERT:
  ''' Класс векторизации текстовых данных.'''

  # Загружаем предобученныею модель токенизации текста
  def __init__(self, tokenizer_path, model_path):
    self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    self.model = AutoModel.from_pretrained(model_path)

  # Приводим текст в его векторное представление
  def vectorize(self, text):
    encoded_input = self.tokenizer(
        [text],
        padding=True,
        truncation=True,
        max_length=24,
        return_tensors='pt')

    with torch.no_grad():
      model_output = self.model(**encoded_input)
    
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    return sentence_embeddings[0]

class ClassificationModel:
  '''
    Модель классификации векторных представлений текста.
    Общий объём тегов классификации: 39
  '''

  dtset_col_txt_name = 'content'
  dtset_col_tag_name = 'tag'

  #
  def __init__(self, vectorizer):
    self.vectorizer = vectorizer

  #
  def load_dataset(self, path):
    self.dataset = pd.read_csv(path,sep=';')

  #
  def vectorize_dataset(self):
    self.vectorized_content = list(
        map(lambda t: self.vectorizer.vectorize(t),
            self.dataset[self.dtset_col_txt_name]))

  #
  def split_dataset(self, d_split=0.4):
    self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
        self.vectorized_content,
        self.dataset[self.dtset_col_tag_name],
        test_size=d_split,
        random_state=0)

  #
  def train_model(self, model_name, params = None, save_dir_path='./'):
    self. model = CatBoostClassifier(
        iterations=30,
        learning_rate=0.1,
        depth=8,
        loss_function='TotalF1',
        average='Macro')
    
    self.model.fit(
        self.X_train,
        self.y_train,
        eval_set=(self.X_val, self.y_val))
    
    self.model.save_model(save_dir_path + model_name)

  #
  def load_model(self, model_path):
    self.model = CatBoostClassifier()
    self.model.load_model(model_path)

  # 
  def predict_tag(self, vector):
    prediction = self.model.predict_proba(vector)
    highest_score = max(zip(TAG_RANGE,prediction), key=lambda pair: pair[1])
    return highest_score

  

# Парсинг документа

In [ ]:
def parse_doc(doc_path):
  pass

Общая инициализация

In [ ]:
vectorizer = TextVectorizerBERT(TOKENIZER, TOKENIZER)
model = ClassificationModel(vectorizer)

#Обучение модели

In [ ]:
model.load_split_dataset(DATASET_NAME)
model.train_model(MODEL_NAME,
                  PARAMS,
                  MODEL_DIR)

## Получение предсказаний модели

In [ ]:
DOC_PATH = ''

In [ ]:
model.load_model(MODEL_NAME)

In [ ]:
# Разбиваем весь документ на абзацы, которые будем классифицировать
texts = parse_doc(DOC_PATH)

In [ ]:
finded_tags = []

# Векторизуем и классифицируем извлечённый абзац документа
for text in texts:
  text_embedding = model.vectorizer.vectorize()
  tag = model.predict_tag(text_embedding)
  # Сохраняем тег, полученный при классификации
  finded_tags.append(tag)

tag_frequency = Counter(finded_tags)

Формирование выходных тегов

In [ ]:
missed_tags = set(TAG_RANGE).difference(set(finded_tags)) 